In [1]:
import pandas as pd
import csv
import os
import matplotlib.pyplot as plt
import seaborn as sns

from utils.analysis.reader import read_results, read_dataset

In [2]:
# configs
metircs = ['mae', 'mape', 'rmse', 'smape']

Dataset = ['JONAS_NYC_bike', 'JONAS_NYC_taxi', 'Metr-LA','METRO_HZ', 'METRO_SH','PEMS03', 'PEMS07',
           'COVID_DEATHS', 'ETT_hour', 'electricity', 'weather', 'Jena_climate', 'nasdaq100']

model_order = ['DCRNN-0', 'DCRNN-2', 'DCRNN-3', 'NET3-0', 'NET3-2', 'AGCRN-0', 'AGCRN-2', 
               'MTGNN-0', 'MTGNN-2', 'ST_Norm-0', 'ST_Norm-2', 'TimesNet-0', 'StemGNN-0',
               'AutoFormer-0', 'CrossFormer-0', 'PatchTST-0']

In [3]:
res_path = '/home/zhuangjiaxin/workspace/TensorTSL/Tensor-Time-Series/output/csv/96-12-merge-v2.csv'
dataset_path = '/home/zhuangjiaxin/workspace/TensorTSL/Tensor-Time-Series/datasets/Tensor-Time-Series-Dataset/Processed_Data/datasummary.csv'
results = read_results(res_path, Dataset)
dataset = read_dataset(dataset_path)

1. 从数据集的类型来区分：按数据集的类别进行分类，选出每个数据集的前三名。

In [4]:
MTS_list = []
TS_list = []
for dataset_name in dataset:
    if dataset[dataset_name]['type'] == 'MTS':
        MTS_list.append(dataset_name)
    elif dataset[dataset_name]['type'] == 'TS':
        TS_list.append(dataset_name)
    else:
        print(f'Error: dataset type error:{dataset[dataset_name]["type"]}')
print(MTS_list)
print(TS_list)

['Metr-LA', 'ETT_hour', 'PEMS20', 'PEMS04', 'PEMS08', 'PEMS03', 'PEMS07', 'PEMSBAY', 'COVID_DEATHS', 'electricity', 'weather', 'Jena_climate']
['JONAS_NYC_taxi', 'JONAS_NYC_bike', 'METRO_HZ', 'COVID_CHI', 'COVID_US', 'METRO_SH', 'stocknet', 'nasdaq100', 'crypto12']


In [37]:
import numpy as np
# MTS vs TS
list_map = {
    'MTS': MTS_list,
    'TS': TS_list,
}
list_name = 'TS'
metric_name = 'mae'
top_k = 10
for dataset_name in list_map[list_name]:
    value_list = []
    model_name_list = []
    for model_name in results:
        if dataset_name in results[model_name]:
            res_item = results[model_name][dataset_name]
            name = model_name
            metric = res_item.metric
            value  = metric[metric_name]
            if value == 'metric_error':
                continue
            value = float(value)
            if np.isnan(value):
                continue
            value_list.append(value)
            model_name_list.append(name)
    if value_list == []:
        continue  
      
    combine = zip(value_list, model_name_list)
    sorted_combine = sorted(combine, key=lambda x: x[0])
    value_list, model_name_list = zip(*sorted_combine)
    # mae_list = list(mae_list)
    value_list = list(value_list)
    model_name_list = list(model_name_list)
    value_list = value_list[:top_k]
    model_name_list = model_name_list[:top_k]
    # 创建条形图
    plt.figure(figsize=(14, 8))
    bars = plt.bar(model_name_list, value_list, color='skyblue')

    # 添加标题和轴标签
    plt.title(f'{metric_name.upper()} for {dataset_name} (TOP {top_k})')
    plt.xlabel('Model')
    plt.ylabel(metric_name.upper())
    plt.xticks(rotation=45)
    for bar in bars:
        height = bar.get_height()
        if np.isfinite(height):
            plt.text(bar.get_x() + bar.get_width() / 2, height, round(height, 2), ha='center', va='bottom')
    plt.savefig(os.path.join('/home/zhuangjiaxin/workspace/TensorTSL/Tensor-Time-Series/output/ipynb_img', f'{metric_name}_{list_name}_{dataset_name}.png'))
    plt.close()
    

    


2. 绘制 datasets 的雷达图

In [5]:
import numpy as np
DatasetMap = {
    "Traffic": ['JONAS_NYC_bike', 'JONAS_NYC_taxi', 'Metr-LA','METRO_HZ', 'METRO_SH','PEMS03', 'PEMS07'],
    "Natural": ['COVID_DEATHS'],
    "Energy":  ['ETT_hour', 'electricity'],
    "Weather": ['weather', 'Jena_climate'],
    "Finance": ['nasdaq100'],
}

In [8]:
labels = ['featrue_corr_mean', 'feature_corr_std', 'dataset_mean_dis', 'dataset_std_dis', 'node_corr_mean', 'node_corr_std']
data = []
names = []
dataset_type = 'Traffic-Finance'
for dataset_name in dataset:
    if dataset_name in DatasetMap['Traffic'] or dataset_name in DatasetMap['Finance']:#or dataset_name in DatasetMap['Energy'] or dataset_name in DatasetMap['Weather']:
        item = dataset[dataset_name]
        data.append([item['feature_corr_mean'], item['feature_corr_std'], item['dataset_mean_dis'], item['dataset_std_dis'], item['node_corr_mean'], item['node_corr_std']])
        names.append(dataset_name)
labels = np.array(labels)
data = np.array(data)
min_data = data.min(axis=0)
max_data = data.max(axis=0)
ranges = max_data - min_data
# data = (data - min_data) / ranges
data = np.nan_to_num(data, nan=0.0)
# print(data)
angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
data = np.concatenate((data, data[:,[0]]), axis=1)
angles += angles[:1]

fig, ax = plt.subplots(figsize=(12, 10), subplot_kw=dict(polar=True))
colors = sns.color_palette("Set1", len(data))
color_pos = 0
for i in data:
    c = colors[color_pos]
    name = names[color_pos]
    ax.fill(angles, i, color=c, alpha=0.3, label=f'{name}')
    ax.plot(angles, i, color=c, linewidth=1)
    color_pos += 1

ax.set_yticklabels([])
ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels, fontsize=12)
ax.legend(loc='right', bbox_to_anchor=(1.2, 0.85))
plt.savefig(os.path.join('/home/zhuangjiaxin/workspace/TensorTSL/Tensor-Time-Series/output/ipynb_img', f'Radar_{dataset_type}.png'))
plt.close()